In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pyBigWig

In [9]:
!head concat_peaks.tsv

chr14	32460567	32461717	cCREs144340
chr11	90722232	90722963	cCREs85014
chr15	21631167	21631680	cCREs164573
chr10	13228856	13229395	cCREs41799
chr16	29904342	29904857	cCREs187773
chr16	55134890	55135443	cCREs193193
chr3	140476498	140477022	cCREs312343
chr12	38001573	38002102	cCREs98818
chr10	34405265	34405852	cCREs46199
chr2	41505885	41506873	cCREs256056


### Load data

In [ ]:
scores_path = "/oak/stanford/groups/akundaje/projects/aav/explain_scores/CLAGL/fold0/split_scores/CLAGL.fold0.explain-all.npy"
scores = np.load(scores_path, allow_pickle = True)

### Choose sequence length

In [ ]:
seqlens = np.array([x.shape[0] for x in scores])
print(stats.describe(seqlens))

In [ ]:
plt.hist(seqlens, bins=30)
plt.show()

In [ ]:
#Almost 96% of the sequence lenghts are less than 1000
print(f"Fraction of sequences shorter than 1000 bp: {np.mean(seqlens<1000)}")

In [ ]:
seqlen = 1000

### Crop and Pad

Sequences shorter than 1000 bp will be zero-padded and sequences longer than 1000 bp will be cropped.

In [ ]:
def crop(arr, seqlen = 1000):
    center = len(arr)//2
    return arr[center-seqlen//2: center+seqlen//2]

def pad(arr, seqlen = 1000):
    padlength = (1000-len(arr))//2
    return np.pad(arr, padlength)

#### Surag script explain scores -> bigwig step by step

In [1]:
scores="/oak/stanford/groups/akundaje/projects/aav/explain_scores/CLAGL/fold0/split_scores/CLAGL.fold0.explain-all.npy"
regions="/oak/stanford/groups/akundaje/projects/aav/explain_inputs/concat_peaks.tsv"
chrom_sizes="/home/groups/akundaje/jelenter/SVM_pipelines/make_inputs/mm10.chrom.sizes"
outfile="/oak/stanford/groups/akundaje/projects/aav/explain_scores/CLAGL/fold0/split_scores/CLAGL.fold0.explain-all.bw"

In [2]:
with open(chrom_sizes) as f:
    gs = [x.strip().split('\t') for x in f]
gs = [(x[0], int(x[1])) for x in gs]

chr_to_idx = {}
for i,x in enumerate(gs):
    chr_to_idx[x[0]] = i

In [5]:
f = np.load(scores, allow_pickle = 1)

with open(regions) as r:
    regions_dup = [x.strip().split('\t') for x in r]
    
regions_dup = [[x[0], int(x[1]), int(x[2])] for x in regions_dup]

In [6]:
regions = []
keep_idxs = []
for n, i in enumerate(regions_dup):
    if i not in regions[:n]:
        regions.append(i)
        keep_idxs.append(n)
f = f[keep_idxs]

In [7]:
# regions may not be sorted, so get their sorted order
order_of_regs = sorted(range(len(regions)), key=lambda x:(chr_to_idx[regions[x][0]], regions[x][1]))

# regions may overlap but as we go in sorted order, we will ignore the values that are repeated 
# and only consider the first instance

bw = pyBigWig.open(outfile, 'w')
bw.addHeader(gs)

all_entries = []
cur_chr = ""
cur_end = 0

iterator = order_of_regs

In [ ]:
for i in iterator:
    print(i)
    
    if regions[i][0]!=cur_chr: 
        cur_chr = regions[i][0]
        cur_end = 0

    # bring current end to at least start of current region
    if cur_end < regions[i][1]:
        cur_end = regions[i][1]

    #assert(regions[i][2]>=cur_end)

    vals = np.sum(f[i], axis=1).astype(np.float64)[cur_end-regions[i][1]:]
    print(vals.shape)
    bw.addEntries([regions[i][0]]*(regions[i][2]-cur_end), 
                   list(range(cur_end,regions[i][2])), 
                   ends = list(range(cur_end+1, regions[i][2]+1)), 
                   values=list(vals))

    all_entries.append(vals)
    
    cur_end = regions[i][2]+1

bw.close()